In [8]:
#Text for adding in inactives
import requests  # pip install requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import statistics as stat
from functools import reduce

In [9]:
date = '02-12-2021'
home_id = 'ATL'
away_id = 'SAS'

In [10]:
# Sets up url to scrape and gets data for home and away boxscores
date_triple = date.split('-')
month = date_triple[0]
day = date_triple[1]
year = date_triple[2]
url = 'https://www.basketball-reference.com/boxscores/' + year + month + day + '0' + home_id + '.html'
# basic boxscore - away
df_away = pd.read_html(url)[0]
df_away = pd.concat([df_away['Unnamed: 0_level_0'], df_away['Basic Box Score Stats']], axis=1, sort=False)
#advanced boxscore - away
df_away_adv = pd.read_html(url)[7]
# clean up advanced boxscore - away
df_away_adv1 = pd.concat([df_away_adv['Unnamed: 0_level_0'],df_away_adv['Advanced Box Score Stats']], axis=1, sort=False)
df_away_adv = df_away_adv1.drop(['Starters','MP'],axis=1)

# basic boxscore - home
df_home = pd.read_html(url)[8]
df_home = pd.concat([df_home['Unnamed: 0_level_0'], df_home['Basic Box Score Stats']], axis=1, sort=False)
# advanced boxscore - home
df_home_adv = pd.read_html(url)[15]
# clean up advanced boxscore - home
df_home_adv1 = pd.concat([df_home_adv['Unnamed: 0_level_0'],df_home_adv['Advanced Box Score Stats']], axis=1, sort=False)
df_home_adv = df_home_adv1.drop(['Starters','MP'],axis=1)
# merge basic and advanced boxscores
df_away_all = pd.concat([df_away, df_away_adv], axis=1, sort=False)
df_home_all = pd.concat([df_home, df_home_adv], axis=1, sort=False)
df_list = [df_away_all, df_home_all]
ids = [away_id, home_id]


In [43]:
data = """
<div>
    <label>Name:</label>
    John Smith
</div>
"""

soup = bs(data, "html.parser")

label = soup.find("label", text="Name:")
print(label)
print(label.next_sibling.strip())


<label>Name:</label>


In [61]:
# Get html text from a url
url_text = requests.get(url).text
#put into beautiful soup
soup = bs(url_text,'lxml')
#print(soup)
test = soup.find_all("strong")

test2 = test.find('strong', text='Inactive:')
print(test)

TypeError: expected string or bytes-like object

In [7]:
for home_away in range(2):
        df = df_list[home_away]

        # Deletes unwanted rows and joins 2 sub-dataframes into a single dataframe
        rows = list(df.index.values)
        last_row = rows[len(rows) - 1]
        df = df.drop([5, last_row])

        # Creates and adds a "Starter" variable (1 if starter, 0 if reserve)
        starter = [0] * df.shape[0]
        for i in range(5):
            starter[i] = 1
        df.insert(1, 'Status', starter)

        # Creates and adds a "Team" (Team ID) and "Home" (Home team or Away team)
        home = [home_away] * df.shape[0]
        team = [ids[home_away]] * df.shape[0]
        df.insert(1, 'Team', team)
        df.insert(2, 'Home', home)

        # Changes "MP" time format from string '00:00' to float 0.0.
        # Did not play will appear as a 2 in staatus column. Did not dress willappear as 3
        rows = list(df.index.values)
        for i in rows:
            if df['MP'][i] == 'Did Not Play':
                df['Status'][i] = 2
            elif df['MP'][i] == 'Did Not Dress':
                df['Status'][i] = 3
            else:
                col_index = df['MP'][i].find(':')
                df['MP'][i] = int(df['MP'][i][:col_index]) + int(df['MP'][i][(col_index+1):])/60
                
        # Replace Did not play and did not dress with zeros
        df = df.replace('Did Not Play', 0)
        df = df.replace('Did Not Dress',0)

        # Changes values in the "+/-" column from strings to integers
        rows = list(df.index.values)
        for i in rows:
            if df['+/-'][i] == '0':
                df['+/-'][i] = 0
            elif df['+/-'][i] == 0:
                df['+/-'][i] = 0
            elif df['+/-'][i][0] == '+':
                df['+/-'][i] = int(df['+/-'][i][1:])
            else:
                df['+/-'][i] = -1 * int(df['+/-'][i][1:])

        # Drops any column that relates to efficiency stats
        df.drop(['FG%', '3P%', 'FT%'], inplace=True, axis=1)
        
        # Add  inactive Players
        
        

                  Starters Team  Home  Status Pos  FPTS         MP FG FGA 3P  \
0             Jakob Poeltl  SAS     0       1   C  45.9  28.266667  7   7  0   
1           Keldon Johnson  SAS     0       1   C  22.9  27.583333  8  11  2   
2            DeMar DeRozan  SAS     0       1  SF  40.2  25.700000  7  11  0   
3          Dejounte Murray  SAS     0       1  SF  33.7  24.500000  7  13  2   
4            Derrick White  SAS     0       1  SG  25.4  16.966667  3   5  2   
6              Patty Mills  SAS     0       0  PG  19.6  20.750000  4   6  3   
7             Drew Eubanks  SAS     0       0  PG  14.4  19.733333  1   2  0   
8            Lonnie Walker  SAS     0       0  PG   9.2  19.300000  4  10  1   
9                 Rudy Gay  SAS     0       0  PG  14.6  18.116667  2  10  1   
10           Devin Vassell  SAS     0       0  PG  10.2  12.266667  1   5  1   
11  Quinndary Weatherspoon  SAS     0       0  PG   2.2  10.100000  1   3  0   
12              Trey Lyles  SAS     0   

/Users/jacobplata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jacobplata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jacobplata/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jacobplata/anaconda3/lib/python3.6/site-packages/ipyke

In [ ]:
# Add fantasy points column
        fp = (df["PTS"].astype(float) + (1.2*df["TRB"].astype(float)) + (1.5*df["AST"].astype(float)) + (3*df["STL"].astype(float)) + (3*df["BLK"].astype(float)) - df["TOV"].astype(float))
        df.insert(4, 'FPTS', fp)
        
        # Add Positions
        dfnames = pd.read_csv(r'/Users/jacobplata/Desktop/pp/NBA_Name-Pos.csv')
        dfs = [df, dfnames] 
        df_final = reduce(lambda left,right: pd.merge(left,right,on='Starters'), dfs)  
        pos = df_final["Pos"]
        df.insert(4, 'Pos', pos)
    
        print(df)

        boxscore_dir = r'/Users/jacobplata/Desktop/pp'
        dates = os.listdir(boxscore_dir)
        if date not in dates:
            os.mkdir(boxscore_dir + '/' + date)

        games = os.listdir(boxscore_dir + '/' + date)
        game_file = away_id + '_' + home_id + '.csv'
        if game_file not in games or home_away == 1:
            with open(boxscore_dir + '/' + date + '/' + away_id + '_' + home_id + '.csv', 'a', encoding="utf-8") as f:
                if home_away == 0:
                    df.to_csv(f, header=True, index=False)
                else:
                    df.to_csv(f, header=False, index=False)